# 1. Thư viện

In [1]:
import os
import re

from dotenv import load_dotenv
from langchain_qdrant import Qdrant
from qdrant_client.models import Filter, FieldCondition, MatchValue
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from apikeys_GEMINI import APIKeyManager

from langchain.load import dumps, loads
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser

from sentence_transformers import SentenceTransformer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

d:\Download_Python\lib\importlib\__init__.py:127: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  return _bootstrap._gcd_import(name[level:], package, level)


In [2]:
load_dotenv()

MODEL_GEMINI = os.getenv("MODEL_GEMIMI")
if MODEL_GEMINI is None:
    raise ValueError("Environment variable MODEL_GEMINI is not set")
elif not MODEL_GEMINI.startswith("models/"):
    MODEL_GEMINI = f"models/{MODEL_GEMINI}"

APIS_GEMINI_LIST = os.getenv('APIS_GEMINI_LIST').split(',')
key_manager = APIKeyManager(APIS_GEMINI_LIST)

URL_QDRANT_3 = os.getenv("URL_QDRANT_3")
API_QDRANT_3 = os.getenv("API_QDRANT_3")

MAX_DOCS_FOR_CONTENT = 5

# 2. Mô hình embedding

## 2.1. Mô hình bkai cho Qdrant

In [3]:
EMBEDDINGS_MODEL_bkai = "bkai-foundation-models/vietnamese-bi-encoder"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}

embeddings_bkai = HuggingFaceBgeEmbeddings(
    model_name=EMBEDDINGS_MODEL_bkai,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

## 2.2. Mô hình paraphrase-multilingual-mpnet-base-v2

In [4]:
MODEL_RERANK = 'sentence-transformers/paraphrase-multilingual-mpnet-base-v2'
rerank_model = SentenceTransformer(MODEL_RERANK)

d:\Download_Python\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


# 3. Kết nối đến Collection

## 3.1. Luat_bkai_Article-Section_More_Keywords

In [5]:
exist_ASMK_Collection = Qdrant.from_existing_collection(
    embedding = embeddings_bkai,
    url = URL_QDRANT_3,
    api_key = API_QDRANT_3,
    prefer_grpc=True,
    collection_name = "Luat_bkai_Article-Section_More_Keywords",
	metadata_payload_key="metadata"
)

## 3.2. Luat_bkai_Article_More_Keywords

In [6]:
exist_AMK_Collection = Qdrant.from_existing_collection(
    embedding = embeddings_bkai,
    url = URL_QDRANT_3,
    api_key = API_QDRANT_3,
    prefer_grpc=True,
    collection_name = "Luat_bkai_Article_More_Keywords",
	metadata_payload_key="metadata"
)

# 4. Hàm sinh ra Similarity Queries

In [7]:
def query_generator(original_query: str, key_manager) -> list[str]:
    """Generate queries from original query"""
    # Câu truy vấn gốc
    query = original_query
    
    # Cập nhật prompt để yêu cầu rõ ràng chỉ trả về 3 câu truy vấn và câu gốc
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", "Bạn là một trợ lý hữu ích và có nhiệm vụ tạo ra nhiều truy vấn tìm kiếm dựa trên một truy vấn gốc."),
            ("human", """Tạo chính xác 3 câu truy vấn tìm kiếm liên quan đến: {original_query}. Mỗi câu truy vấn trên một dòng mới. 
            Không được trả về nhiều hơn hoặc ít hơn 3 câu truy vấn. Đảm bảo không thêm bất kỳ văn bản nào khác ngoài 3 câu truy vấn này."""),
        ]
    )
    
    model = ChatGoogleGenerativeAI(
        # key api google gemini, nếu test mà bị báo lỗi api core thì lấy api khác trong .env để test
        google_api_key=key_manager.get_next_key(),
        model=MODEL_GEMINI,
        temperature=0.15
    )
    
    query_generator_chain = (
        prompt | model | StrOutputParser()
    )
    
    # Kết quả sẽ là một chuỗi các câu truy vấn cách nhau bằng dấu xuống dòng
    result = query_generator_chain.invoke({"original_query": query})
    
    # Tách kết quả thành danh sách các câu truy vấn
    generated_queries = result.strip().split('\n')
    
    # Đảm bảo chỉ lấy 3 câu truy vấn nếu có nhiều hơn 3 câu sinh ra
    if len(generated_queries) > 3:
        generated_queries = generated_queries[:3]
    
    # Kết hợp câu gốc với các câu truy vấn sinh ra
    queries = [query] + generated_queries
    
    return queries

# 5. Tách Keywords

## 5.1. Tách keywords

In [8]:
# Hàm trích xuất từ khóa từ văn bản, loại bỏ dấu câu ngoại trừ "/" và "-"
def extract_keywords(text):
    text = re.sub(r'[^\w\s/-]', '', text)  # Loại bỏ các dấu câu không mong muốn
    words = text.split()                   # Tách các từ theo khoảng trắng
    unique_words = list(dict.fromkeys(words))  # Loại bỏ từ trùng lặp
    return unique_words

## 5.2. Xử lý lower case

In [9]:
# Hàm xử lý các từ khóa: chuyển về chữ thường, giữ nguyên số La Mã và từ khóa có "/" hoặc "-"
def process_keywords(keywords):
    result = []
    roman_numeral_pattern = re.compile(r"^(?=[MDCLXVI])M{0,4}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})$")
    
    for word in keywords:
        if roman_numeral_pattern.match(word) or any(c in word for c in "/-"):
            result.append(word)
        else:
            result.append(word.lower())  # Chuyển các từ còn lại thành chữ thường
    return result

# 6. Truy vấn

## 6.1. Hàm print_Results

In [10]:
def print_Results_MoreKeywords(results):
    # for doc,score in results:
    #     print("Page_Content:\n",doc.page_content)
    #     # print("Context:\n",doc.metadata['context'],"\n")
        # print("Stt:",doc.metadata['stt'])
        # print("Số hiệu:",doc.metadata['so_hieu'])
        # print("Chủ đề:",doc.metadata['chu_de'])
        # print("Chương:",doc.metadata["Chapter"])
        # print("Mục:",doc.metadata["Section"])
        # print("Tiểu mục:",doc.metadata["Mini-Section"])
        # print("Điều:",doc.metadata["Article"])
        # print("Khoản:",doc.metadata["Article-Section"])
    #     print("Score:",score,"\n","-----------------------")
    for result in results:
        doc = result[0]  # lấy phần tử đầu tiên
        score_Qdrant = result[1]  # lấy phần tử thứ hai
        score_Rerank = result[2]
        print("Page_Content:\n", doc.page_content)
        print("Stt:",doc.metadata['stt'])
        print("Số hiệu:",doc.metadata['so_hieu'])
        print("Chủ đề:",doc.metadata['chu_de'])
        print("Chương:",doc.metadata["Chapter"])
        print("Mục:",doc.metadata["Section"])
        print("Tiểu mục:",doc.metadata["Mini-Section"])
        print("Điều:",doc.metadata["Article"])
        print("Khoản:",doc.metadata["Article-Section"])
        print("Page_Rerank:", score_Rerank)
        print("Score Qdrant:",score_Qdrant,"\n","-----------------------")

## 6.2. Tạo Filter

In [11]:
def create_should_filter(user_keywords, metadata_fields):
    should_conditions = []

    for keyword in user_keywords:
        for field in metadata_fields:
            should_conditions.append(FieldCondition(
                key=field, 
                match=MatchValue(value=keyword)
            ))

    # Trả về bộ lọc với các điều kiện `should`
    return Filter(
        should=should_conditions
    )

## 6.3. Truy vấn với Filter

In [12]:
def search_documents_with_should_filter(user_query, metadata_fields, top_k=5):
    # Tách keywords từ query của user
    user_keywords = process_keywords(extract_keywords(user_query))
    
    # Tạo bộ lọc `should`
    filter_conditions = create_should_filter(user_keywords, metadata_fields)
    
    # Thực hiện tìm kiếm trên Qdrant với filter `should`
    search_results = exist_ASMK_Collection.similarity_search_with_score(
        query=user_query,
        filter=filter_conditions,
        k=top_k
    )
    
    return search_results

In [13]:
def combined_search(user_query: str, key_manager, metadata_fields, top_k=5):
    # Gọi hàm query_generator để sinh ra 3 truy vấn từ query gốc
    queries = query_generator(user_query, key_manager)

    print("4 câu queries:\n")
    for q in queries:
        print(q)

    print("\nCác kết quả trả về:\n")
    # Lưu trữ kết quả cho từng query
    query_results = []

    # Thực hiện tìm kiếm cho mỗi query trong danh sách queries
    for query in queries:
        search_results = search_documents_with_should_filter(query, metadata_fields, top_k=top_k)
        query_results.extend(search_results)  # Lưu kết quả riêng cho từng query

    # Dictionary để lưu các kết quả unique, key là `doc.page_content`
    unique_results = {}

    # Duyệt qua từng kết quả
    for doc, score in query_results:
        # Kiểm tra nếu `doc.page_content` đã tồn tại trong unique_results
        if doc.page_content in unique_results:
            # Nếu tồn tại, so sánh score và giữ lại cái có score cao hơn
            if score > unique_results[doc.page_content][1]:
                unique_results[doc.page_content] = (doc, score)
        else:
            # Nếu chưa tồn tại, thêm vào unique_results
            unique_results[doc.page_content] = (doc, score)

    # Trả về danh sách kết quả duy nhất, với các giá trị từ dictionary
    return list(unique_results.values())

## 6.4. Re-rank theo trọng số Similarity và TF-IDF

In [14]:
def calculate_scores(user_query, unique_results):
    # Tạo danh sách các doc.page_content từ unique_results
    documents = [doc.page_content for doc, _ in unique_results]
    
    # Tính cosine similarity
    user_query_embedding = rerank_model.encode(user_query)
    document_embeddings = rerank_model.encode(documents)
    cosine_similarities = cosine_similarity([user_query_embedding], document_embeddings)[0]

    # Tính TF-IDF
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform([user_query] + documents)
    tfidf_scores = tfidf_matrix.toarray()[0][1:]  # Lấy chỉ số TF-IDF cho các documents

    # Tính điểm tổng hợp
    combined_scores = []
    for cos_sim, tfidf_score in zip(cosine_similarities, tfidf_scores):
        combined_score = 0.7 * cos_sim + 0.3 * tfidf_score
        combined_scores.append(combined_score)

    # Tạo danh sách kết quả với điểm số
    results_with_scores = [(unique_results[i][0], unique_results[i][1], combined_scores[i]) for i in range(len(unique_results))]

    # Sắp xếp kết quả theo điểm số giảm dần
    results_with_scores.sort(key=lambda x: x[2], reverse=True)

    # Lấy top 5 kết quả
    top_results = results_with_scores[:5]
    
    return top_results

## 6.5. Các metadata cần filter

In [15]:
# Các metadata fields cần lọc
metadata_fields = [
    "metadata.loai_van_ban_Keywords", 
    # "metadata.noi_ban_hanh_Keywords", 
    "metadata.so_hieu", 
    "metadata.linhvuc_nganh_Keywords", 
    "metadata.ngay_ban_hanh", 
    "metadata.ngay_hieu_luc", 
    "metadata.chu_de_Keywords", 
    # "metadata.key_words", 
    # "metadata.Chapter_Keywords", 
    # "metadata.Section_Keywords",  
    # "metadata.Mini-Section_Keywords", 
    "metadata.Article_Keywords", 
    "metadata.Content_Keywords", 
    # "metadata.Article-Section_Keywords"
]

## 6.6. Thực hiện truy vấn Khoản

In [21]:
# Ví dụ sử dụng
user_query= "Nguyên tắc trong khám, chữa bệnh là gồm những nguyên tắc nào?"

search_Results = combined_search(user_query, key_manager, metadata_fields, top_k=5)

re_Rank_Results = calculate_scores(user_query, search_Results)

# In ra kết quả top 5 tài liệu
print_Results_MoreKeywords(re_Rank_Results)

4 câu queries:

Nguyên tắc trong khám, chữa bệnh là gồm những nguyên tắc nào?
Nguyên tắc cơ bản trong khám chữa bệnh
Các nguyên tắc y đức trong khám chữa bệnh
4 nguyên tắc đạo đức trong y học là gì

Các kết quả trả về:

Page_Content:
 5. Tuân thủ quy tắc đạo đức nghề nghiệp trong hành nghề khám bệnh, chữa bệnh do Bộ trưởng Bộ Y tế ban hành.
Stt: 6
Số hiệu: 15/2023/QH15
Chủ đề: Khám bệnh, chữa bệnh
Chương: Chương I: NHỮNG QUY ĐỊNH CHUNG
Mục: None
Tiểu mục: None
Điều: Điều 3. Nguyên tắc trong khám bệnh, chữa bệnh
Khoản: Khoản 5
Page_Rerank: 0.47609505057334894
Score Qdrant: 0.380840927362442 
 -----------------------
Page_Content:
 3. Căn cứ quy định của Luật này và quy định khác của pháp luật có liên quan, người đứng đầu cơ sở khám bệnh, chữa bệnh quy định và tổ chức triển khai các biện pháp bảo đảm an ninh, trật tự cho cơ sở khám bệnh, chữa bệnh và an toàn cho người hành nghề, người khác làm việc tại cơ sở khám bệnh, chữa bệnh, người ra vào cơ sở khám bệnh, chữa bệnh.
Stt: 6
Số hiệu: 1

## 6.7. Lấy metadata để truy xuất Điều

In [22]:
def extract_unique_metadata(top_results):
    metadata_list = []
    metadata_dict_set = set()  # Sử dụng set để lưu trữ các metadata duy nhất

    # Truy cập vào từng result trong top_results
    for result in top_results:
        doc = result[0]  # Lấy doc từ result
        
        # Tạo một dictionary chứa các thuộc tính từ metadata
        metadata = {
            "stt": doc.metadata.get('stt'),
            "loai_van_ban": doc.metadata.get('loai_van_ban'),
            "noi_ban_hanh": doc.metadata.get('noi_ban_hanh'),
            "so_hieu": doc.metadata.get('so_hieu'),
            "linhvuc_nganh": doc.metadata.get('linhvuc_nganh'),
            "ngay_ban_hanh": doc.metadata.get('ngay_ban_hanh'),
            "ngay_hieu_luc": doc.metadata.get('ngay_hieu_luc'),
            "chu_de": doc.metadata.get('chu_de'),
            "Chapter": doc.metadata.get('Chapter'),
            "Section": doc.metadata.get('Section'),
            "Mini-Section": doc.metadata.get('Mini-Section'),
            "Article": doc.metadata.get('Article'),
        }

        # Lọc bỏ các key-value có giá trị None
        filtered_metadata = {key: value for key, value in metadata.items() if value is not None}

        # Chuyển đổi dict thành tuple để thêm vào set
        metadata_tuple = tuple(filtered_metadata.items())
        
        # Kiểm tra và thêm vào set nếu chưa có
        metadata_dict_set.add(metadata_tuple)

    # Chuyển đổi lại set thành list và định dạng lại thành dict
    for metadata_tuple in metadata_dict_set:
        metadata_dict = dict(metadata_tuple)
        metadata_list.append(metadata_dict)

    return metadata_list

In [23]:
list_Metadata = extract_unique_metadata(re_Rank_Results)
print(list_Metadata)

[{'stt': '6', 'loai_van_ban': 'Luật', 'noi_ban_hanh': 'Quốc hội', 'so_hieu': '15/2023/QH15', 'linhvuc_nganh': 'Thể thao - Y tế', 'ngay_ban_hanh': '09/01/2023', 'ngay_hieu_luc': '19/02/2023', 'chu_de': 'Khám bệnh, chữa bệnh', 'Chapter': 'Chương X: ĐIỀU KIỆN BẢO ĐẢM HOẠT ĐỘNG KHÁM BỆNH, CHỮA BỆNH', 'Section': 'Mục 3. CÁC ĐIỀU KIỆN BẢO ĐẢM KHÁC', 'Article': 'Điều 114. Bảo đảm an ninh, trật tự tại cơ sở khám bệnh, chữa bệnh'}, {'stt': '6', 'loai_van_ban': 'Luật', 'noi_ban_hanh': 'Quốc hội', 'so_hieu': '15/2023/QH15', 'linhvuc_nganh': 'Thể thao - Y tế', 'ngay_ban_hanh': '09/01/2023', 'ngay_hieu_luc': '19/02/2023', 'chu_de': 'Khám bệnh, chữa bệnh', 'Chapter': 'Chương I: NHỮNG QUY ĐỊNH CHUNG', 'Article': 'Điều 3. Nguyên tắc trong khám bệnh, chữa bệnh'}, {'stt': '6', 'loai_van_ban': 'Luật', 'noi_ban_hanh': 'Quốc hội', 'so_hieu': '15/2023/QH15', 'linhvuc_nganh': 'Thể thao - Y tế', 'ngay_ban_hanh': '09/01/2023', 'ngay_hieu_luc': '19/02/2023', 'chu_de': 'Khám bệnh, chữa bệnh', 'Chapter': 'Chương 

## 6.8. Thực hiện truy vấn Điều

In [24]:
def search_documents_with_metadata_filter(list_Metadata, top_k=1):
    search_results = []

    # Duyệt qua từng phần tử trong list_Metadata
    for metadata in list_Metadata:        
        # Thực hiện tìm kiếm với query trống và bộ lọc
        results = exist_AMK_Collection.similarity_search_with_score(
            query="",  # Query để trống
            filter=metadata,
            k=top_k
        )
        
        # Thêm kết quả vào danh sách tìm kiếm
        search_results.extend(results)

    return search_results

In [27]:
article_Results = search_documents_with_metadata_filter(list_Metadata)

for doc, score in article_Results:
    print("Stt:",doc.metadata['stt'],"\n")
    print("Điều và Nội dung Điều:\n",doc.metadata["combine_Article_Content"],"\n","-----------------------")
    # print(re)

Stt: 6 

Điều và Nội dung Điều:
 Điều 114. Bảo đảm an ninh, trật tự tại cơ sở khám bệnh, chữa bệnh
1. Các biện pháp bảo đảm an ninh, trật tự cho cơ sở khám bệnh, chữa bệnh và an toàn cho người hành nghề, người khác làm việc tại cơ sở khám bệnh, chữa bệnh bao gồm:
a) Thiết lập hệ thống giám sát và cảnh báo nguy cơ gây mất an ninh, trật tự;
b) Giới hạn ra vào cơ sở khám bệnh, chữa bệnh theo giờ và khu vực;
c) Ứng dụng các giải pháp công nghệ cao để quản lý người bệnh, thân nhân của người bệnh, người hành nghề để tăng cường an ninh, trật tự đối với cơ sở khám bệnh, chữa bệnh;
d) Có biện pháp phù hợp để bảo quản tài sản cho người bệnh và thân nhân của người bệnh trong quá trình khám bệnh, chữa bệnh tại cơ sở khám bệnh, chữa bệnh;
đ) Không cho phép đưa vào cơ sở khám bệnh, chữa bệnh các chất, phương tiện, công cụ, vật dụng gây mất an ninh, trật tự cho cơ sở khám bệnh, chữa bệnh;
e) Biện pháp bảo đảm an ninh, trật tự khác theo quy định của pháp luật.
2. Trong trường hợp người bệnh, thân nhân